In [1]:
from google.colab import drive


In [2]:
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
import os
# Change the current directory
os.chdir('/content/drive/MyDrive/GPANet-main/GPANet')

In [4]:
!pip install scikit-image


In [ ]:
"""
Different utilities such as orthogonalization of weights, initialization of
loggers, etc

Copyright (C) 2018, Matias Tassano <matias.tassano@parisdescartes.fr>

This program is free software: you can use, modify and/or
redistribute it under the terms of the GNU General Public
License as published by the Free Software Foundation, either
version 3 of the License, or (at your option) any later
version. You should have received a copy of this license along
this program. If not, see <http://www.gnu.org/licenses/>.
"""
import subprocess
import math
import logging
import numpy as np
import cv2
import torch
import torch.nn as nn
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

from copy import copy
import torch.nn.functional as F

def weights_init_kaiming(lyr):
	r"""Initializes weights of the model according to the "He" initialization
	method described in "Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification" - He, K. et al. (2015), using a
    normal distribution.
	This function is to be called by the torch.nn.Module.apply() method,
	which applies weights_init_kaiming() to every layer of the model.
	"""
	classname = lyr.__class__.__name__
	if classname.find('Conv') != -1:
		nn.init.kaiming_normal_(lyr.weight.data, a=0, mode='fan_in')
	elif classname.find('Linear') != -1:
		nn.init.kaiming_normal_(lyr.weight.data, a=0, mode='fan_in')
	elif classname.find('BatchNorm') != -1:
		lyr.weight.data.normal_(mean=0, std=math.sqrt(2./9./64.)).\
			clamp_(-0.025, 0.025)
		nn.init.constant_(lyr.bias.data, 0.0)

def batch_psnr(img, imclean, data_range):
	r"""
	Computes the PSNR along the batch dimension (not pixel-wise)

	Args:
		img: a `torch.Tensor` containing the restored image
		imclean: a `torch.Tensor` containing the reference image
		data_range: The data range of the input image (distance between
			minimum and maximum possible values). By default, this is estimated
			from the image data-type.
	"""
	img_cpu = img.data.cpu().numpy().astype(np.float32)
	imgclean = imclean.data.cpu().numpy().astype(np.float32)
	psnr = 0
	for i in range(img_cpu.shape[0]):
		psnr += compare_psnr(imgclean[i, :, :, :], img_cpu[i, :, :, :], \
					   data_range=data_range)
	return psnr/img_cpu.shape[0]

def data_augmentation(image, mode):
	r"""Performs dat augmentation of the input image

	Args:
		image: a cv2 (OpenCV) image
		mode: int. Choice of transformation to apply to the image
			0 - no transformation
			1 - flip up and down
			2 - rotate counterwise 90 degree
			3 - rotate 90 degree and flip up and down
			4 - rotate 180 degree
			5 - rotate 180 degree and flip
			6 - rotate 270 degree
			7 - rotate 270 degree and flip
	"""
	out = np.transpose(image, (1, 2, 0))
	if mode == 0:
		# original
		out = out
	elif mode == 1:
		# flip up and down
		out = np.flipud(out)
	elif mode == 2:
		# rotate counterwise 90 degree
		out = np.rot90(out)
	elif mode == 3:
		# rotate 90 degree and flip up and down
		out = np.rot90(out)
		out = np.flipud(out)
	elif mode == 4:
		# rotate 180 degree
		out = np.rot90(out, k=2)
	elif mode == 5:
		# rotate 180 degree and flip
		out = np.rot90(out, k=2)
		out = np.flipud(out)
	elif mode == 6:
		# rotate 270 degree
		out = np.rot90(out, k=3)
	elif mode == 7:
		# rotate 270 degree and flip
		out = np.rot90(out, k=3)
		out = np.flipud(out)
	else:
		raise Exception('Invalid choice of image transformation')
	return np.transpose(out, (2, 0, 1))

def variable_to_cv2_image(varim):
	r"""Converts a torch.autograd.Variable to an OpenCV image

	Args:
		varim: a torch.autograd.Variable
	"""
	nchannels = varim.size()[1]
	if nchannels == 1:
		res = (varim.data.cpu().numpy()[0, 0, :]*255.).clip(0, 255).astype(np.uint8)
	elif nchannels == 3:
		res = varim.data.cpu().numpy()[0]
		res = cv2.cvtColor(res.transpose(1, 2, 0), cv2.COLOR_RGB2BGR)
		res = (res*255.).clip(0, 255).astype(np.uint8)
	else:
		raise Exception('Number of color channels not supported')
	return res

def get_git_revision_short_hash():
	r"""Returns the current Git commit.
	"""
	return subprocess.check_output(['git', 'rev-parse', '--short', 'HEAD']).strip()

def init_logger(argdict):
	r"""Initializes a logging.Logger to save all the running parameters to a
	log file

	Args:
		argdict: dictionary of parameters to be logged
	"""
	from os.path import join

	logger = logging.getLogger(__name__)
	logger.setLevel(level=logging.INFO)
	fh = logging.FileHandler(join(argdict.log_dir, 'log.txt'), mode='a')
	formatter = logging.Formatter('%(asctime)s - %(message)s')
	fh.setFormatter(formatter)
	logger.addHandler(fh)
	try:
		logger.info("Commit: {}".format(get_git_revision_short_hash()))
	except Exception as e:
		logger.error("Couldn't get commit number: {}".format(e))
	logger.info("Arguments: ")
	for k in argdict.__dict__:
		logger.info("\t{}: {}".format(k, argdict.__dict__[k]))

	return logger

def init_logger_ipol():
	r"""Initializes a logging.Logger in order to log the results after
	testing a model

	Args:
		result_dir: path to the folder with the denoising results
	"""
	logger = logging.getLogger('testlog')
	logger.setLevel(level=logging.INFO)
	fh = logging.FileHandler('out.txt', mode='w')
	formatter = logging.Formatter('%(message)s')
	fh.setFormatter(formatter)
	logger.addHandler(fh)

	return logger

def init_logger_test(result_dir):
	r"""Initializes a logging.Logger in order to log the results after testing
	a model

	Args:
		result_dir: path to the folder with the denoising results
	"""
	from os.path import join

	logger = logging.getLogger('testlog')
	logger.setLevel(level=logging.INFO)
	fh = logging.FileHandler(join(result_dir, 'log.txt'), mode='a')
	formatter = logging.Formatter('%(asctime)s - %(message)s')
	fh.setFormatter(formatter)
	logger.addHandler(fh)

	return logger

def normalize(data):
	r"""Normalizes a unit8 image to a float32 image in the range [0, 1]

	Args:
		data: a unint8 numpy array to normalize from [0, 255] to [0, 1]
	"""
	return np.float32(data/255.)

def svd_orthogonalization(lyr):
	r"""Applies regularization to the training by performing the
	orthogonalization technique described in the paper "FFDNet:	Toward a fast
	and flexible solution for CNN based image denoising." Zhang et al. (2017).
	For each Conv layer in the model, the method replaces the matrix whose columns
	are the filters of the layer by new filters which are orthogonal to each other.
	This is achieved by setting the singular values of a SVD decomposition to 1.

	This function is to be called by the torch.nn.Module.apply() method,
	which applies svd_orthogonalization() to every layer of the model.
	"""
	classname = lyr.__class__.__name__
	if classname.find('Conv') != -1:
		weights = lyr.weight.data.clone()
		c_out, c_in, f1, f2 = weights.size()
		dtype = lyr.weight.data.type()

		# Reshape filters to columns
		# From (c_out, c_in, f1, f2)  to (f1*f2*c_in, c_out)
		weights = weights.permute(2, 3, 1, 0).contiguous().view(f1*f2*c_in, c_out)

		# Convert filter matrix to numpy array
		weights = weights.cpu().numpy()

		# SVD decomposition and orthogonalization
		mat_u, _, mat_vh = np.linalg.svd(weights, full_matrices=False)
		weights = np.dot(mat_u, mat_vh)

		# As full_matrices=False we don't need to set s[:] = 1 and do mat_u*s
		lyr.weight.data = torch.Tensor(weights).view(f1, f2, c_in, c_out).\
			permute(3, 2, 0, 1).type(dtype)
	else:
		pass

def remove_dataparallel_wrapper(state_dict):
	r"""Converts a DataParallel model to a normal one by removing the "module."
	wrapper in the module dictionary

	Args:
		state_dict: a torch.nn.DataParallel state dictionary
	"""
	from collections import OrderedDict

	new_state_dict = OrderedDict()
	for k, vl in state_dict.items():
		name = k[7:] # remove 'module.' of DataParallel
		new_state_dict[name] = vl

	return new_state_dict

def is_rgb(im_path):
	r""" Returns True if the image in im_path is an RGB image
	"""
	from skimage.io import imread
	rgb = False
	im = imread(im_path)
	if (len(im.shape) == 3):
		if not(np.allclose(im[...,0], im[...,1]) and np.allclose(im[...,2], im[...,1])):
			rgb = True
	print("rgb: {}".format(rgb))
	print("im shape: {}".format(im.shape))
	return rgb




In [21]:
!python test_demo.py

  File "/content/drive/MyDrive/GPANet-main/GPANet/test_demo.py", line 128
    img_g = img_g / 255.0 
                          ^
IndentationError: unindent does not match any outer indentation level


In [33]:
# -*- coding: utf-8 -*-
"""
Created on Sat Mar 21 20:48:05 2020
@author: Administrator
"""

import torch
import torch.nn as nn
import numpy as np
import cv2
import time
import os
from model import *
import utils_train

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def load_checkpoint(checkpoint_dir, IsGPU):
    if IsGPU == 1:
        model_info = torch.load(checkpoint_dir + 'checkpoint.pth.tar')
        net = Main()
        device_ids = [0]
        model = nn.DataParallel(net, device_ids=device_ids).cuda()
        model.load_state_dict(model_info['state_dict'])
        optimizer = torch.optim.Adam(model.parameters())
        optimizer.load_state_dict(model_info['optimizer'])
        cur_epoch = model_info['epoch']
    else:
        model_info = torch.load(checkpoint_dir + 'checkpoint.pth.tar', map_location=torch.device('cpu'))
        net = Main()
        device_ids = [0]
        model = nn.DataParallel(net, device_ids=device_ids)
        model.load_state_dict(model_info['state_dict'])
        optimizer = torch.optim.Adam(model.parameters())
        optimizer.load_state_dict(model_info['optimizer'])
        cur_epoch = model_info['epoch']
    return model, optimizer, cur_epoch

def adjust_learning_rate(optimizer, epoch, lr_update_freq):
    if not epoch % lr_update_freq and epoch:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1
            print(param_group['lr'])
    return optimizer

def FSobel_XY(data):
    data = cv2.cvtColor(chw_to_hwc(data), cv2.COLOR_BGR2GRAY)
    x = cv2.Sobel(np.clip(data*255, 0, 255).astype('uint8'), cv2.CV_8U, 0, 1, ksize=3)
    absX = cv2.convertScaleAbs(x)
    y = cv2.Sobel(np.clip(data*255, 0, 255).astype('uint8'), cv2.CV_8U, 1, 0, ksize=3)
    absY = cv2.convertScaleAbs(y)
    dst = cv2.addWeighted(absX, 0.5, absY, 0.5, 0)
    return dst / 255.0

def FLap(data):
    data = cv2.cvtColor(chw_to_hwc(data), cv2.COLOR_BGR2GRAY)
    x = cv2.GaussianBlur(data, (3,3), 0)
    x = cv2.Laplacian(np.clip(x*255, 0, 255).astype('uint8'), cv2.CV_8U, ksize=3)
    Lap = cv2.convertScaleAbs(x)
    return Lap / 255.0

def GFSobel_XY(data):
    x = cv2.Sobel(np.clip(data*255, 0, 255).astype('uint8'), cv2.CV_8U, 0, 1, ksize=3)
    absX = cv2.convertScaleAbs(x)
    y = cv2.Sobel(np.clip(data*255, 0, 255).astype('uint8'), cv2.CV_8U, 1, 0, ksize=3)
    absY = cv2.convertScaleAbs(y)
    dst = cv2.addWeighted(absX, 0.5, absY, 0.5, 0)
    return dst / 255.0

def GFLap(data):
    x = cv2.GaussianBlur(data, (3,3), 0)
    x = cv2.Laplacian(np.clip(x*255, 0, 255).astype('uint8'), cv2.CV_8U, ksize=3)
    Lap = cv2.convertScaleAbs(x)
    return Lap / 255.0

def hwc_to_chw(img):
    return np.transpose(img, axes=[2, 0, 1])

def chw_to_hwc(img):
    return np.transpose(img, axes=[1, 2, 0])

if __name__ == '__main__':
    checkpoint_dir = './checkpoint/'
    test_dir = '/content/drive/MyDrive/GPANet-main/GPANet/dataset/low/'
    result_dir = '/content/drive/MyDrive/GPANet-main/GPANet/dataset/Test_result/LOL'
    testfiles = os.listdir(test_dir)
    IsGPU = 1    #GPU is 1, CPU is 0

    print('> Loading dataset ...')

    lr_update_freq = 30
    model, optimizer, cur_epoch = load_checkpoint(checkpoint_dir, IsGPU)

    if IsGPU == 1:
        for f in range(len(testfiles)):
            model.eval()
            with torch.no_grad():
                img = cv2.imread(test_dir + testfiles[f])
                img_g = cv2.imread(test_dir + testfiles[f], 0)
                img_g = img_g / 255.0
                h, w, c = img.shape
                img_ccc = img / 255.0
                img_h = hwc_to_chw(img_ccc)
                input_var = torch.from_numpy(img_h.copy()).type(torch.FloatTensor).unsqueeze(0).cuda()
                input_svar = torch.from_numpy(GFSobel_XY(img_g.copy())).type(torch.FloatTensor).unsqueeze(0).unsqueeze(0).cuda()
                input_lapvar = torch.from_numpy(GFLap(img_g.copy())).type(torch.FloatTensor).unsqueeze(0).unsqueeze(0).cuda()
                s = time.time()
                lapout, lbpout, e_out = model(input_var, input_svar, input_lapvar)
                e = time.time()
                print(input_var.shape)
                print(e-s)
                e_out = e_out.squeeze().cpu().detach().numpy()
                e_out = chw_to_hwc(e_out)
                e_out = cv2.resize(e_out, (w, h))
                newlap = np.zeros((e_out.shape))
                newlbp = np.zeros((e_out.shape))
                lap_out = lapout.squeeze().squeeze().cpu().detach().numpy()
                lap_out = cv2.resize(lap_out, (w, h))
                for clap in range(3):
                    newlap[:,:,clap] = lap_out
                lbp_out = lbpout.squeeze().squeeze().cpu().detach().numpy()
                lbp_out = cv2.resize(lbp_out, (w, h))
                for clbp in range(3):
                    newlbp[:,:,clbp] = lbp_out

                temp = e_out

                print("Saving result for:", result_dir + '/' + testfiles[f][:-4] + '_Grad4' + '.png')

                cv2.imwrite(result_dir + '/' + testfiles[f][:-4] + '_Grad4' + '.png', np.clip(temp * 255, 0.0, 255.0))


> Loading dataset ...
torch.Size([1, 3, 800, 600])
0.0373539924621582
Saving result for: /content/drive/MyDrive/GPANet-main/GPANet/dataset/Test_result/LOL/146_Grad4.png
torch.Size([1, 3, 800, 600])
0.011958122253417969
Saving result for: /content/drive/MyDrive/GPANet-main/GPANet/dataset/Test_result/LOL/669_Grad4.png
torch.Size([1, 3, 800, 600])
0.01083517074584961
Saving result for: /content/drive/MyDrive/GPANet-main/GPANet/dataset/Test_result/LOL/79_Grad4.png
torch.Size([1, 3, 800, 600])
0.012266159057617188
Saving result for: /content/drive/MyDrive/GPANet-main/GPANet/dataset/Test_result/LOL/22_Grad4.png
torch.Size([1, 3, 800, 600])
0.01016545295715332
Saving result for: /content/drive/MyDrive/GPANet-main/GPANet/dataset/Test_result/LOL/778_Grad4.png
torch.Size([1, 3, 800, 600])
0.010848045349121094
Saving result for: /content/drive/MyDrive/GPANet-main/GPANet/dataset/Test_result/LOL/179_Grad4.png
torch.Size([1, 3, 800, 600])
0.010802984237670898
Saving result for: /content/drive/MyDriv

In [11]:
import os
from PIL import Image

# Directory containing the images
input_dir = "/content/drive/MyDrive/LOLdataset/eval15/low/"
output_dir = "/content/drive/MyDrive/GPANet-main/GPANet/dataset/low/"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Iterate over all files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust file extensions as needed
        # Open the image
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path)

        # Resize the image to 800x600
        resized_img = img.resize((600, 800))

        # Save the resized image to the output directory
        output_path = os.path.join(output_dir, filename)
        resized_img.save(output_path)

        print(f"Resized {filename} successfully.")


Resized 146.png successfully.
Resized 669.png successfully.
Resized 79.png successfully.
Resized 22.png successfully.
Resized 778.png successfully.
Resized 179.png successfully.
Resized 55.png successfully.
Resized 547.png successfully.
Resized 1.png successfully.
Resized 493.png successfully.
Resized 665.png successfully.
Resized 748.png successfully.
Resized 780.png successfully.
Resized 23.png successfully.
Resized 111.png successfully.


In [15]:
import os
from PIL import Image

# Directory containing the images
input_dir = "/content/drive/MyDrive/LOLdataset/eval15/high/"
output_dir = "/content/drive/MyDrive/GPANet-main/GPANet/Clear/"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Iterate over all files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust file extensions as needed
        # Open the image
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path)

        # Resize the image to 800x600
        resized_img = img.resize((600, 800))

        # Save the resized image to the output directory
        output_path = os.path.join(output_dir, filename)
        resized_img.save(output_path)

        print(f"Resized {filename} successfully.")


Resized 146.png successfully.
Resized 669.png successfully.
Resized 79.png successfully.
Resized 22.png successfully.
Resized 778.png successfully.
Resized 179.png successfully.
Resized 55.png successfully.
Resized 1.png successfully.
Resized 547.png successfully.
Resized 493.png successfully.
Resized 748.png successfully.
Resized 780.png successfully.
Resized 665.png successfully.
Resized 23.png successfully.
Resized 111.png successfully.


In [17]:
import torch
import torch.nn as nn
import numpy as np
import cv2
import time
import os
from model import *
import utils_train

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def load_checkpoint(checkpoint_dir, IsGPU):
    if IsGPU == 1:
        model_info = torch.load(checkpoint_dir + 'checkpoint.pth.tar')
        net = Main()
        device_ids = [0]
        model = nn.DataParallel(net, device_ids=device_ids).cuda()
        model.load_state_dict(model_info['state_dict'])
        optimizer = torch.optim.Adam(model.parameters())
        optimizer.load_state_dict(model_info['optimizer'])
        cur_epoch = model_info['epoch']
    else:
        model_info = torch.load(checkpoint_dir + 'checkpoint.pth.tar', map_location=torch.device('cpu'))
        net = Main()
        device_ids = [0]
        model = nn.DataParallel(net, device_ids=device_ids)
        model.load_state_dict(model_info['state_dict'])
        optimizer = torch.optim.Adam(model.parameters())
        optimizer.load_state_dict(model_info['optimizer'])
        cur_epoch = model_info['epoch']

    return model, optimizer, cur_epoch

def adjust_learning_rate(optimizer, epoch, lr_update_freq):
    if not epoch % lr_update_freq and epoch:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1
            print(param_group['lr'])
    return optimizer

def train_psnr(train_in, train_out):
    psnr = utils_train.batch_psnr(train_in, train_out, 1.)
    return psnr

def FSobel_XY(data):
    data = cv2.cvtColor(chw_to_hwc(data), cv2.COLOR_BGR2GRAY)
    x = cv2.Sobel(np.clip(data*255, 0, 255).astype('uint8'), cv2.CV_8U, 0, 1, ksize=3)
    absX = cv2.convertScaleAbs(x)
    y = cv2.Sobel(np.clip(data*255, 0, 255).astype('uint8'), cv2.CV_8U, 1, 0, ksize=3)
    absY = cv2.convertScaleAbs(y)
    dst = cv2.addWeighted(absX, 0.5, absY, 0.5, 0)
    return dst / 255.0

def FLap(data):
    data = cv2.cvtColor(chw_to_hwc(data), cv2.COLOR_BGR2GRAY)
    x = cv2.GaussianBlur(data, (3, 3), 0)
    x = cv2.Laplacian(np.clip(x*255, 0, 255).astype('uint8'), cv2.CV_8U, ksize=3)
    Lap = cv2.convertScaleAbs(x)
    return Lap / 255.0

def GFSobel_XY(data):
    x = cv2.Sobel(np.clip(data*255, 0, 255).astype('uint8'), cv2.CV_8U, 0, 1, ksize=3)
    absX = cv2.convertScaleAbs(x)
    y = cv2.Sobel(np.clip(data*255, 0, 255).astype('uint8'), cv2.CV_8U, 1, 0, ksize=3)
    absY = cv2.convertScaleAbs(y)
    dst = cv2.addWeighted(absX, 0.5, absY, 0.5, 0)
    return dst / 255.0

def GFLap(data):
    x = cv2.GaussianBlur(data, (3, 3), 0)
    x = cv2.Laplacian(np.clip(x*255, 0, 255).astype('uint8'), cv2.CV_8U, ksize=3)
    Lap = cv2.convertScaleAbs(x)
    return Lap / 255.0

def hwc_to_chw(img):
    return np.transpose(img, axes=[2, 0, 1])

def chw_to_hwc(img):
    return np.transpose(img, axes=[1, 2, 0])

if __name__ == '__main__':
    checkpoint_dir = './checkpoint/'
    test_dir = '/content/drive/MyDrive/GPANet-main/GPANet/dataset/low/'
    #test_dir_1 = '/content/drive/MyDrive/LOLdataset/eval15/high/'
    result_dir = './Test_result'
    testfiles = os.listdir(test_dir)
    testfiles_1 = os.listdir(test_dir_1)
    IsGPU = 1    # GPU is 1, CPU is 0

    print('> Loading dataset ...')

    lr_update_freq = 30
    model, optimizer, cur_epoch = load_checkpoint(checkpoint_dir, IsGPU)

    if IsGPU == 1:
        for f in range(len(testfiles)):
            model.eval()
            with torch.no_grad():
                img = cv2.imread(test_dir  +'/'+ testfiles[f])
                img_g = cv2.imread(test_dir + '/' + testfiles[f],0)

                img_g = img_g / 255.0
                h, w, c = img.shape
                img_ccc = img / 255.0
                img_h = hwc_to_chw(img_ccc)
                input_var = torch.from_numpy(img_h.copy()).type(torch.FloatTensor).unsqueeze(0).cuda()

                input_svar = torch.from_numpy(GFSobel_XY(img_g.copy())).type(torch.FloatTensor).unsqueeze(0).cuda()
                input_lapvar = torch.from_numpy(GFLap(img_g.copy())).type(torch.FloatTensor).unsqueeze(0).cuda()

                s = time.time()
                lapout, lbpout, e_out = model(input_var, input_svar, input_lapvar)
                e = time.time()
                print(input_var.shape)
                print(e-s)

                e_out = e_out.squeeze().cpu().detach().numpy()
                e_out = chw_to_hwc(e_out)
                e_out = cv2.resize(e_out, (w, h))

                newlap = np.zeros((e_out.shape))
                newlbp = np.zeros((e_out.shape))

                lap_out = lapout.squeeze().squeeze().cpu().detach().numpy()
                lap_out = cv2.resize(lap_out, (w, h))

                for clap in range(3):
                    newlap[:, :, clap] = lap_out

                lbp_out = lbpout.squeeze().squeeze().cpu().detach().numpy()
                lbp_out = cv2.resize(lbp_out, (w, h))

                for clbp in range(3):
                    newlbp[:, :, clbp] = lbp_out

                temp = e_out  # np.concatenate((img/255,e_out,newlap, newlbp), axis=1)

                cv2.imwrite(result_dir + '/' + testfiles[f][:-4] + '_Grad4' + '.png', np.clip(temp*255, 0.0, 255.0))


> Loading dataset ...


RuntimeError: Tensors must have same number of dimensions: got 4 and 3